In [ ]:
import marimo as mo

In [ ]:
import random
import pyscipopt as scip
from ortools.sat.python import cp_model

# 1 機械リリース時刻付き重み付き完了時刻和最小化問題

- 機械: 1 つだけ
- ジョブ $J = \{ 1, \dots, n \}$
- 各ジョブの処理時間: $p_j \ (\forall j \in J)$
- 各ジョブの重要度: $w_j \ (\forall j \in J)$
- 各ジョブのリリース時刻: $r_j \ (\forall j \in J)$
- 各ジョブの処理完了時刻: $C_j \ (\forall j \in J)$

$C_j$ の重み付き和を最小化する.

## 離接定式化(Disjunctive formulation)

$M$ を大きな定数として

\begin{align*}
&\text{minimize} &\sum_{j=1}^n w_j s_j + \sum_{j=1}^n w_j p_j \\
&\text{s.t.} &s_j + p_j - M (1 - x_{jk}) &\leq s_k \ &(\forall j \neq k) \\
& &x_{jk} + x_{kj} &= 1 \ &(\forall j < k) \\
& &s_j &\geq r_j \ &(\forall j \in J) \\
& &x_{jk} &\in \{0, 1\} \ &(\forall j \neq k)
\end{align*}

- 決定変数
    - $s_j$: ジョブ $j$ の開始時刻
    - $x_{jk}$: ジョブ $j$ がジョブ $k$ に先行するとき $1$
- 補足
    - 目的関数の第 2 項目は定数であるため第 1 項だけを最小化すればよい

## 実装

In [ ]:
def make_data(n):
    """
    Data generator for the one machine scheduling problem.
    """

    random.seed(0)
    p, r, d, w = {}, {}, {}, {}

    J = range(1, n + 1)

    for j in J:
        p[j] = random.randint(1, 4)
        w[j] = random.randint(1, 3)

    T = sum(p)
    for j in J:
        r[j] = random.randint(0, 5)
        d[j] = r[j] + random.randint(0, 5)

    return J, p, r, d, w

In [ ]:
class ModelDisjunctiveScip:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.scip = scip.Model("scheduling: disjunctive")

        # Big M
        M = max(self.r.values()) + sum(self.p.values())

        # start time variable, x[j,k] = 1 if job j precedes job k, 0 otherwise
        self.x = {}
        self.s = {}

        ub = sum(self.p[j] for j in self.J)
        for j in self.J:
            self.s[j] = self.scip.addVar(lb=r[j], ub=ub, vtype="C", name=f"s[{j}]")
            for k in self.J:
                if j != k:
                    self.x[j, k] = self.scip.addVar(vtype="B", name=f"x[{j},{k}]")

        for j in self.J:
            for k in self.J:
                if j != k:
                    self.scip.addCons(
                        self.s[j] - self.s[k] + M * self.x[j, k] <= (M - self.p[j]), f"Bound[{j},{k}]"
                    )

                if j < k:
                    self.scip.addCons(self.x[j, k] + self.x[k, j] == 1, f"Disjunctive[{j},{k}]")

        self.scip.setObjective(scip.quicksum(self.w[_j] * self.s[_j] for _j in self.J), sense="minimize")

    def solve(self) -> None:
        self.scip.optimize()

    def get_z(self):
        return self.scip.getObjVal() + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.scip.getVal(self.s[_j]) + 0.5), _j) for _j in self.s])]

In [ ]:
class _MyInterval:
    # [start, end)
    def __init__(self, scip: scip.Model, lb: int, ub: int, size: int):
        self.lb = lb
        self.ub = ub
        self.size = size
        self.start = scip.addVar(lb=self.lb, ub=self.ub - self.size, vtype='C')
        self.end = self.start + self.size

def _my_add_no_overlap(scip: scip.Model, jobs: dict[int, _MyInterval]):
    if len(jobs) == 0:
        return
    for _j1 in jobs.keys():
        for _j2 in jobs.keys():
            if _j2 <= _j1:
                continue
            big_m = max(jobs[_j1].ub - jobs[_j2].lb, jobs[_j2].ub - jobs[_j1].lb)
            tmp1 = scip.addVar(vtype='B')
            scip.addCons(
                jobs[_j1].end <= jobs[_j2].start + big_m * (1 - tmp1)
            )
            tmp2 = scip.addVar(vtype='B')
            scip.addCons(
                jobs[_j2].end <= jobs[_j1].start + big_m * (1 - tmp2)
            )
            scip.addCons(tmp1 + tmp2 >= 1)

class ModelIntervalScip:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.scip = scip.Model("scheduling: disjunctive")

        ub = sum(p[_j] for _j in J)
        self.jobs = {_j: _MyInterval(self.scip, self.r[_j], ub, self.p[_j]) for _j in self.J}
        _my_add_no_overlap(self.scip, self.jobs)

        self.scip.setObjective(scip.quicksum(self.w[_j] * self.jobs[_j].start for _j in self.J), sense="minimize")

    def solve(self) -> None:
            self.scip.optimize()

    def get_z(self):
        return self.scip.getObjVal() + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.scip.getVal(self.jobs[_j].start) + 0.5), _j) for _j in self.jobs])]

In [ ]:
class ModelDisjunctiveCpSat:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.cp = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        # Big M
        M = max(r.values()) + sum(p.values())

        # start time variable, x[j,k] = 1 if job j precedes job k, 0 otherwise
        self.x = {}
        self.s = {}

        ub = sum(p[j] for j in J)
        for j in J:
            self.s[j] = self.cp.new_int_var(lb=r[j], ub=ub, name=f"s[{j}]")
            for k in J:
                if j != k:
                    self.x[j, k] = self.cp.new_bool_var(name=f"x[{j},{k}]")

        for j in J:
            for k in J:
                if j != k:
                    self.cp.add(self.s[j] - self.s[k] + M * self.x[j, k] <= (M - p[j]))

                if j < k:
                    self.cp.add(self.x[j, k] + self.x[k, j] == 1)

        self.cp.minimize(sum(self.w[j] * self.s[j] for j in J))

    def solve(self) -> None:
        self.solver.parameters.log_search_progress = True
        self.solver.solve(self.cp)

    def get_z(self):
        return self.solver.objective_value + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.solver.value(self.s[_j]) + 0.5), _j) for _j in self.s])]

In [ ]:
class ModelIntervalCpSat:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.cp = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        ub = sum(p[_j] for _j in J)
        s = {_j: self.cp.new_int_var(lb=self.r[_j], ub=ub, name=f"s[{_j}]") for _j in self.J}
        self.jobs = {_j: self.cp.new_fixed_size_interval_var(s[_j], self.p[_j], f"jobs[{_j}]") for _j in self.J}
        self.cp.add_no_overlap(self.jobs.values())

        self.cp.minimize(sum(self.w[_j] * self.jobs[_j].start_expr() for _j in J))

    def solve(self) -> None:
        self.solver.parameters.log_search_progress = True
        self.solver.solve(self.cp)

    def get_z(self):
        return self.solver.objective_value + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(self.solver.value(self.jobs[_j].start_expr()), _j) for _j in self.jobs])]

In [ ]:
n = 30
J, p, r, d, w = make_data(n)

In [ ]:
run_scip_disj = mo.ui.run_button(label="Run", full_width=True)
run_scip_disj

&lt;marimo-button data-initial-value=&#x27;0&#x27; data-label=&#x27;&amp;quot;&amp;lt;span class=&amp;#92;&amp;quot;markdown prose dark:prose-invert&amp;#92;&amp;quot;&amp;gt;&amp;lt;span class=&amp;#92;&amp;quot;paragraph&amp;#92;&amp;quot;&amp;gt;Run&amp;lt;/span&amp;gt;&amp;lt;/span&amp;gt;&amp;quot;&#x27; data-kind=&#x27;&amp;quot;neutral&amp;quot;&#x27; data-disabled=&#x27;false&#x27; data-full-width=&#x27;true&#x27;&gt;&lt;/marimo-button&gt;

In [ ]:
if run_scip_disj.value:
    _model = ModelDisjunctiveScip(J, p, r, w)
    with mo.redirect_stderr():
        _model.solve()

    _z = _model.get_z()
    _seq = _model.get_seq()
    mo.md(f"""
    - Opt.value by Disjunctive Formulation: {_z}
    - Solution: {_seq}
    """)

In [ ]:
run_scip_interval = mo.ui.run_button(label="Run", full_width=True)
run_scip_interval

&lt;marimo-button data-initial-value=&#x27;0&#x27; data-label=&#x27;&amp;quot;&amp;lt;span class=&amp;#92;&amp;quot;markdown prose dark:prose-invert&amp;#92;&amp;quot;&amp;gt;&amp;lt;span class=&amp;#92;&amp;quot;paragraph&amp;#92;&amp;quot;&amp;gt;Run&amp;lt;/span&amp;gt;&amp;lt;/span&amp;gt;&amp;quot;&#x27; data-kind=&#x27;&amp;quot;neutral&amp;quot;&#x27; data-disabled=&#x27;false&#x27; data-full-width=&#x27;true&#x27;&gt;&lt;/marimo-button&gt;

In [ ]:
if run_scip_interval.value:
    _model = ModelIntervalScip(J, p, r, w)
    with mo.redirect_stderr():
        _model.solve()

    _z = _model.get_z()
    _seq = _model.get_seq()
    mo.md(f"""
    - Optimal value: {_z}
    - Solution: {_seq}
    """)

In [ ]:
_model = ModelDisjunctiveCpSat(J, p, r, w)
with mo.redirect_stderr():
    _model.solve()

_z = _model.get_z()
_seq = _model.get_seq()
mo.md(f"""
- Opt.value by Disjunctive Formulation: {_z}
- Solution: {_seq}
""")


Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x89d2d6e0295b344b)
#Variables: 900 (#ints: 30 in objective)
  - 870 Booleans in [0,1]
  - 5 in [0,74]
  - 4 in [1,74]
  - 9 in [2,74]
  - 4 in [3,74]
  - 2 in [4,74]
  - 6 in [5,74]
#kLinear2: 435
#kLinear3: 870

Starting presolve at 0.00s
  1.42e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.88e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  8.25e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.79e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.33e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'234 nodes and 3'074 arcs.
[Symmetry] Symmetry computation done. time: 0.00020009 dtime: 0.00044344
  1.38e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.40e-03s  5.07e-04d  [Probe] #probed=1'740 
  4.32e-06s  0.00e+00d  [MaxClique] 
  1.32e-04

  1.37e-04s  0.00e+00d  [DetectDominanceRelations] 
  9.39e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.19e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.35e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'265 nodes and 3'134 arcs.
[Symmetry] Symmetry computation done. time: 0.00020016 dtime: 0.00044797
  1.45e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  6.21e-03s  6.03e-04d  [Probe] #probed=1'740 
  2.56e-06s  0.00e+00d  [MaxClique] 
  1.40e-04s  0.00e+00d  [DetectDominanceRelations] 
  9.54e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  9.84e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.34e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.18e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.25e-06s  0.00e+00d  [DetectDominatedLinearConstraints] 
  1.19e-04s  0.00e+00d  [DetectDifferentVariables] 
  4.60e-06s  0.00e+00d  [P


Presolved optimization model '': (model_fingerprint: 0x11ab455e049cc254)
#Variables: 465 (#ints: 30 in objective)
  - 435 Booleans in [0,1]
  - 5 in [0,74]
  - 4 in [1,74]
  - 9 in [2,74]
  - 4 in [3,74]
  - 2 in [4,74]
  - 6 in [5,74]
#kInterval: 30
#kLinear2: 870 (#enforced: 870)
#kNoOverlap: 1 (#intervals: 30)
[Symmetry] Graph for symmetry has 1'830 nodes and 3'134 arcs.
[Symmetry] Symmetry computation done. time: 0.000190611 dtime: 0.00042187

Preloading model.
#Bound   0.02s best:inf   next:[157,4736] initial_domain
#Model   0.02s var:465/465 constraints:901/901

Starting search at 0.02s with 12 workers.
8 full problem subsolvers: [core, default_lp, fixed, max_lp, no_lp, quick_restart, quick_restart_no_lp, reduced_costs]
4 first solution subsolvers: [fj(2), fs_random, fs_random_no_lp]
13 interleaved subsolvers: [feasibility_pump, graph_arc_lns, graph_cst_lns, graph_dec_lns, graph_var_lns, ls, ls_lin, rins/rens, rnd_cst_lns, rnd_var_lns, scheduling_intervals_lns, scheduling_preced

#1       0.04s best:1650  next:[157,1649] fixed
#2       0.04s best:1618  next:[157,1617] fixed
#3       0.04s best:1615  next:[157,1614] fixed


#Bound   0.06s best:1615  next:[159,1614] quick_restart_no_lp
#4       0.06s best:1609  next:[159,1608] quick_restart_no_lp
#Bound   0.06s best:1609  next:[1208,1608] max_lp


...

#Model  44.14s var:425/465 constraints:828/901


#Model  44.39s var:423/465 constraints:826/901


#Model  45.07s var:422/465 constraints:824/901


#Done   45.34s max_lp



Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                        'core':         1 [  45.36s,   45.36s]   45.36s   0.00ns   45.36s         1 [  59.00s,   59.00s]   59.00s   0.00ns   59.00s
                  'default_lp':         1 [  45.32s,   45.32s]   45.32s   0.00ns   45.32s         1 [  14.79s,   14.79s]   14.79s   0.00ns   14.79s
            'feasibility_pump':         1 [  6.17ms,   6.17ms]   6.17ms   0.00ns   6.17ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                       'fixed':         1 [  45.32s,   45.32s]   45.32s   0.00ns   45.32s         1 [  32.94s,   32.94s]   32.94s   0.00ns   32.94s
                          'fj':         1 [ 63.43ms,  63.43ms]  63.43ms   0.00ns  63.43ms         1 [ 95.29ms,  95.29ms]  95.29ms   0.00ns  95.29ms
                          'fj':         1 [ 65.53ms,  65.53ms]  65.53ms   0.00ns  65.53ms         1 [100.25ms, 

In [ ]:
_model = ModelIntervalCpSat(J, p, r, w)
with mo.redirect_stderr():
    _model.solve()

_z = _model.get_z()
_seq = _model.get_seq()
mo.md(f"""
- Optimal value: {_z}
- Solution: {_seq}
""")


Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x77d9b8aa7f03b1b1)
#Variables: 30 (#ints: 30 in objective)
  - 5 in [0,74]
  - 4 in [1,74]
  - 9 in [2,74]
  - 4 in [3,74]
  - 2 in [4,74]
  - 6 in [5,74]
#kInterval: 30
#kNoOverlap: 1 (#intervals: 30)

Starting presolve at 0.00s


  1.37e-05s  0.00e+00d  [DetectDominanceRelations] 
  5.41e-05s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.15e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.78e-06s  0.00e+00d  [DetectDuplicateColumns] 
  1.02e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 61 nodes and 60 arcs.
[Symmetry] Symmetry computation done. time: 1.8335e-05 dtime: 8.17e-06
[Symmetry] #generators: 2, average support size: 2
[Symmetry] 2 orbits on 4 variables with sizes: 2,2
[Symmetry] Found orbitope of size 1 x 2
  9.58e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.72e-04s  1.10e-07d  [Probe] 
  1.92e-06s  0.00e+00d  [MaxClique] 
  7.92e-06s  0.00e+00d  [DetectDominanceRelations] 
  3.25e-05s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  5.71e-07s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  9.22e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  8.23e-05s  0.00e+00d  [DetectDuplicateCons

#Bound   0.01s best:1603  next:[1364,1602] max_lp_sym
#Bound   0.02s best:1603  next:[1376,1602] max_lp_sym
#7       0.02s best:1596  next:[1376,1595] quick_restart_no_lp
#8       0.02s best:1573  next:[1376,1572] quick_restart_no_lp
#9       0.02s best:1571  next:[1376,1570] quick_restart_no_lp


#Bound   0.02s best:1571  next:[1382,1570] max_lp_sym
#10      0.02s best:1567  next:[1382,1566] quick_restart_no_lp
#11      0.03s best:1566  next:[1382,1565] quick_restart_no_lp
#12      0.03s best:1563  next:[1382,1562] quick_restart_no_lp


#13      0.04s best:1561  next:[1382,1560] quick_restart_no_lp
#14      0.05s best:1560  next:[1382,1559] quick_restart


...

#45      0.74s best:1478  next:[1447,1477] graph_dec_lns (d=5.00e-01 s=16 t=0.10 p=0.00 stall=0 h=base) [combined with: quick_restart...]


#46      0.83s best:1476  next:[1447,1475] quick_restart
#47      0.83s best:1475  next:[1447,1474] quick_restart


#Bound   0.86s best:1475  next:[1448,1474] max_lp_sym [skipped_logs=10]
#48      1.02s best:1473  next:[1448,1472] rins_lp_lns (d=2.93e-01 s=27 t=0.10 p=0.00 stall=1 h=base)


#49      1.22s best:1472  next:[1471,1471] max_lp_sym
#Done    1.22s max_lp_sym
#Bound   1.21s best:1473  next:[1471,1472] max_lp_sym [skipped_logs=6]



Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                        'core':         1 [   1.23s,    1.23s]    1.23s   0.00ns    1.23s         1 [   1.22s,    1.22s]    1.22s   0.00ns    1.22s
                  'default_lp':         1 [   1.22s,    1.22s]    1.22s   0.00ns    1.22s         1 [545.78ms, 545.78ms] 545.78ms   0.00ns 545.78ms
            'feasibility_pump':         1 [101.83us, 101.83us] 101.83us   0.00ns 101.83us         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                       'fixed':         1 [   1.22s,    1.22s]    1.22s   0.00ns    1.22s         1 [946.22ms, 946.22ms] 946.22ms   0.00ns 946.22ms
                          'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                          'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,  